<a href="https://colab.research.google.com/github/caduAa/MPLC-/blob/main/MPLC_mys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.fft import fft2, ifft2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# For the field propagation
def split_step(U, H):
  U_fft = np.fft.fft2(U);                                                       #Field's Fourier Transform
  U_propfft = U_fft * H;                                                        # Propagation in Fourier Space
  return np.fft.ifft2(U_propfft);                                               # Back to Real Space

## For initial fields
def gaussian_beam(w0, region):
  j_k = 1j * k;
  z0 = 0.5* k * ((w0)**2);                                                      #Rayleigh range
  W = w0 * np.sqrt(1 + (zList / z0) **2);                                       #Beam waist
  i = 0;
  X,Y = region;
  RHO = np.sqrt(X**2+Y**2);

  R = np.zeros(b);
  for z in zList:
    if z == 0:
      pass
    else:
      R[i] = z * (1 + (z0 / z) **2);
      i = i + 1;

  j_zeta = -1j * z0;
  A0 = 1.;
  A = (w0 / W);
  WW = W ** 2.;

  U = A0 * np.exp(-RHO**2 / W[0] ** 2);

  return U;

def laguerre_gauss_beam(w0, region, l, m):

  j_k = 1j * k;
  z0 = 0.5* k * ((w0)**2);                                                      #Rayleigh range
  W = w0 * np.sqrt(1 + (zList / z0) **2);                                       #Beam waist
  i = 0;
  X,Y = region;
  RHO = np.sqrt(X**2+Y**2);
  PHI = np.arctan(Y/X);

  R = np.zeros(b);
  for z in zList:
    if z == 0:
      pass
    else:
      R[i] = z * (1 + (z0 / z) **2);
      i = i + 1;

  j_zeta = -1j * z0;
  Oz = np.arctan(zList/z0);
  A0 = 1.;
  A = (w0 / W);
  WW = W ** 2.;

  U = A0 * ((RHO / W[0]) ** l) * np.exp(-RHO ** 2 / W[0] ** 2) * np.exp(- (j_k * zList[0]) - 1j * l * PHI + 1j * (l + 2*m + 1) * Oz[0]);                   #terminar

  return U;

#Optimizing...
def optimizing_phases(U1, U2, region, num_iterations, numsteps):

  numsteps = 250;
  tolerance = 1.e-3;
  phases = np.array([np.zeros_like(U1) for step in range(numsteps)]);
  phases_grad = np.array([np.zeros_like(U1) for step in range(numsteps)]);
  phases_masks = phases.shape[0]
  t1 = np.array([np.zeros_like(U1) for step in range(numsteps)]);
  t2 = np.array([np.zeros_like(U1) for step in range(numsteps)]);
  a = np.array([np.zeros_like(U1) for step in range(numsteps)]);

  Vol_U1 = np.array([np.ones_like(U1) for step in range(numsteps)]);
  Vol_U1 = Vol_U1 * U1;
  Vol_U2 = np.array([np.ones_like(U2) for step in range(numsteps)]);
  Vol_U2 = Vol_U2 * U2;

  u1 = Vol_U1;
  u2 = Vol_U2;

  k = 0;
  i = 0;

  for k in range(1, numsteps):

    t1 = np.exp(1j * phases[k - 1]);
    t2 = np.exp(1j * phases[-(k + 1)]);
    Vol_U1[k] = split_step(Vol_U1[k - 1] * t1, H);
    Vol_U2[-(k + 1)] = split_step(Vol_U2[-k] * t2, np.conj(H));

  E0 = np.real(np.mean(np.abs(Vol_U1-Vol_U2)**2));

  if (E0 <= tolerance):
    return phases, E0;
  else:
    for i in range(num_iterations):

      phases_grad = np.angle(np.conj(Vol_U1) * Vol_U2);
      a = phases_grad;
      k = 0;
      Vol_U1 = u1;
      Vol_U2 = u2;

      for k in range(1, numsteps):

        t1 = np.exp(1j * phases_grad[k - 1]);
        t2 = np.exp(1j * phases_grad[-(k + 1)]);
        Vol_U1[k] = split_step(Vol_U1[k - 1] * t1, H);
        Vol_U2[-(k + 1)] = split_step(Vol_U2[-k] * t2, np.conj(H));
        phases_grad = np.angle(np.conj(Vol_U1) * Vol_U2);

      E = np.real(np.mean(np.abs(Vol_U1 - Vol_U2) ** 2));

      if (E < E0):
        #phases = phases_grad;
        E0 = E;
        if (E0 <= tolerance):
          phases = a;
          return phases, E0;
        else:
            break

  return phases, E0;


##Parameters
l=np.linspace(-10,+10,50);
X, Y = np.meshgrid(l,l);
N = 350;
L = 80;
n0 = 1.5078;
lambda_ = 640e-3;
k0 = 2*np.pi/lambda_;
k = k0*n0;

RHO = np.sqrt(X**2+Y**2);

dx, dy = X[1] - X[0], Y[1] - Y[0];
dz = 20;
zFinal = 5000;
Nz = np.round(zFinal/dz);
Nzint = int(Nz);
zList = np.linspace(0,zFinal, Nzint);
b = zList.size;                                                                 #Number of layers (and steps)
pmlWidth = L/8;
insideIndex = np.round(N*[pmlWidth/L, 1-pmlWidth/L]);
region = (insideIndex[0],insideIndex[1]);


#For Fourier Transform
k_x, k_y = 2. * np.pi * np.fft.fftfreq(50, dx), 2. * np.pi * np.fft.fftfreq(50, dy);
K_x, K_y = np.meshgrid(k_x, k_y);
H = np.exp(-1j *dz * np.sqrt((k)**2-(K_x)**2-(K_y)**2));


##Initial Beam conditions...
w0 = 10.;                               #Beam Waist
beamDist = 0.0;
U1 = gaussian_beam(w0, region = (X,Y));
U2 = laguerre_gauss_beam(w0, region = (X,Y), l = 1, m = 0);

phases, E_sqr = optimizing_phases(U1, U2, region = (X,Y), num_iterations = 10, numsteps = 250);

#plt.imshow(np.abs(U1) ** 2., extent=[l[0],l[-1],l[0],l[-1]]);




/usr/local/lib/python3.10/dist-packages/numpy/fft/helper.py:162: RuntimeWarning: divide by zero encountered in divide
  val = 1.0 / (n * d)
/usr/local/lib/python3.10/dist-packages/numpy/fft/helper.py:169: RuntimeWarning: invalid value encountered in multiply
  return results * val
<ipython-input-2-888de37d84ab>:159: RuntimeWarning: invalid value encountered in sqrt
  H = np.exp(-1j *dz * np.sqrt((k)**2-(K_x)**2-(K_y)**2));
<ipython-input-2-888de37d84ab>:90: ComplexWarning: Casting complex values to real discards the imaginary part
  Vol_U1[k] = split_step(Vol_U1[k - 1] * t1, H);
<ipython-input-2-888de37d84ab>:114: ComplexWarning: Casting complex values to real discards the imaginary part
  Vol_U1[k] = split_step(Vol_U1[k - 1] * t1, H);


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.animation as animation

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff()

fig, ax = plt.subplots();
graph = ax.imshow(phases[0])

def integrate(i):
  graph.set_data(phases[i]);

animation.FuncAnimation(fig, integrate, frames= phases_masks, interval=(2000./phases_masks))